# 2주차 — 기상·기후 데이터 전처리


## 학습 목표
- 결측/이상치 처리, 파생지표(누적강우/엽면수분) 생성
- 바이어스 보정 개념 훑어보기(실습은 관측 일자료 중심)

In [ ]:

import pandas as pd, numpy as np, matplotlib.pyplot as plt
from pathlib import Path
from utils import rolling_leaf_wetness_hours

weather = pd.read_csv(str(Path("..")/"data"/"sample_weather_2022.csv"))
weather['date'] = pd.to_datetime(weather['date'])

# 파생 지표
weather['prcp_3d_sum'] = weather['prcp_mm'].rolling(3, min_periods=1).sum()
weather['leaf_wet_3d_sum'] = rolling_leaf_wetness_hours(weather['leaf_wetness_hours'], window=3)

# 시각화
plt.figure(figsize=(10,4))
plt.plot(weather['date'], weather['prcp_3d_sum'])
plt.title("3일 누적 강우")
plt.xlabel("날짜"); plt.ylabel("mm")
plt.tight_layout(); plt.show()
